# A New Programming Paradigm : from rules to data

In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

프로그래밍 언어를 통해 Rule을 코딩하고 data를 입력해 그에 따른 answer를 얻어내는 프로그래밍 방식으로는 풀기 어려운 문제들도 많이 있다. 예를 들어, 이와 같은 rule-based 프로그래밍은 컴퓨터비전 문제를 풀기가 몹시 어렵다. 걷는 것, 뛰는 것, 자전거 타는 것과 같은 무수한 경우의 수들이 존재하므로 이미지를 이 경우의 수들에 대응시키는 rule 모두를 다 코딩하는 것은 불가능하기 때문이다.

그 대신 충분히 많은 data를 입력시켰을 때 암시적으로 내재된 rule을 학습하는 시스템을 프로그래밍한다면, 그 시스템이 학습해 낸 rule을 가지고 문제를 해결할 수 있게 된다. Tensorflow는 Google에서 개발한 'rule을 스스로 찾아내는 시스템'이다. 아래 예시는 Tensorflow를 더 쉽게 사용할 수 있게 해주는 Keras라는 API를 사용하여 아래의 rule을 찾아내는 과정을 보여준다:

$$
y = 50 + 40x
$$

이를 만족하는 데이터를 아래와 같이 생성하고 네트워크를 정의한다.

In [3]:
xs = np.array([0, 2, 3, 6, 8, 5], dtype='float32')
ys = 50 + xs * 40

model = keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])]) 
# Sequential : defines a SEQUENCE of layers in the neural network
# Dense      : adds a layer of neurons

그리고 이 네트워크를 최적화할 방법을 `compile` method를 통해 정하고, 이 네트워크에 `fit` method를 사용해서 parameter들을 적합시킨다

In [4]:
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=1000, verbose=0) 
# epocchs = 1000 : iterate 500 optimizing procedure for 1000 time
# verbose = 0    : fit the network without status message. default is 1

훈련된 네트워크를 사용해서 x = 10.0일 때 y의 값을 예측할 수 있다.

In [5]:
print(model.predict([10.0]))  # i.e. 50 + 40 * 10

[[450.08466]]


# Introduction to Computer Vision

이는 위와 같은 단순한 데이터에만 적용되는 시스템은 아니다. fashion_MNIST라는 데이터셋은 28x28 크기의 의류, 신발 사진을 70,000장 포함하고 있다. 각 데이터는 항목의 이름을 의미하는 label과 짝지어져 있다.

In [6]:
(training_images, training_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

아래와 같은 코드를 통해 이와 같은 이미지를 받아 이를 한 label로 분류하는 rule을 찾아내는 신경망을 훈련시킬 수 있다. 먼저 이 값들이 0에서 255 사이의 값을 가짐을 확인할 수 있는데, 다양한 이유 때문에 이를 0에서 1사이의 값으로 정규화시키는 것이 좋다.

In [7]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

위의 toy example과 똑같은 절차를 똑같은 method들을 사용해 신경망을 구축하고 훈련시킨다. 네트워크를 정의하고 → 이를 compile하고 → 훈련시킨다

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5, verbose=0)

그랬을 때, test set에 대한 성능은 아래와 같이 측정된다.

In [9]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 80us/step


[0.35019856153726575, 0.8746]

## Callback

여기서 나아가 training set에 parameter들이 적당히 적합되었다고 판단될 때 학습을 멈추게 하고 싶을 수도 있다. Callback은 이 경우에 사용하고, 특정 수준의 accuracy가 달성되면 training을 멈추게 하는 기능을 한다. 이 '특정 수준'은 사용자가 지정해야 하는 불편함은 있다.

In [10]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc') > 0.91):
            # If training accuracy reached 0.91, stop training
            self.model.stop_training = True

신경망을 fitting하는 과정에서 이 callback 정보를 입력해주면 된다.

In [11]:
callbacks = myCallback()
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

Epoch 1/5
60000/60000 [==============================] - 6s 100us/step - loss: 0.2815 - acc: 0.8962
Epoch 2/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.2704 - acc: 0.8996
Epoch 3/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.2583 - acc: 0.9038
Epoch 4/5
60000/60000 [==============================] - 6s 102us/step - loss: 0.2495 - acc: 0.9071
Epoch 5/5
60000/60000 [==============================] - 7s 111us/step - loss: 0.2416 - acc: 0.9103


# Enhancing vision with CNN

28x28 크기의 image를 단순히 flatten하여 신경망의 input layer로 사용하는 것에서 나아가 28x28의 정보를 효과적으로 압축해주는 단계를 추가해줌으로써 향상된 성능을 보이는 네트워크를 적합시킬 수 있다. Convolution layer, Pooling layer를 추가하여 이 단계를 구현한다. 이 layer들은 4d-array를 요구하기 때문에 먼저 이에 맞게 데이터의 형태를 조정해준다.

In [15]:
training_images = training_images.reshape(60000, 28, 28, 1)
test_images = test_images.reshape(10000, 28, 28, 1)

그 다음은 똑같다. 네트워크를 정의하고 → 이를 compile하고 → 훈련시킨다.

In [16]:
model = tf.keras.models.Sequential([
    # Adding Convolution-Pooling phase to effectively condense the information
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Using compressed information, fit plain neural network
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5, verbose=0)

이에 더해 네트워크의 모양을 확인해보려면 summary method를 사용한다.

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total para